In [1]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.mappers import ParityMapper
import dill
import numpy as np
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit.circuit.library import EfficientSU2

In [2]:
BL = 0.6

In [3]:
# Parameters
BL = 0.8


### 1 Construct the Hamiltonian

Helper function to construct the Hamiltonian represented as a Pauli decomposition stored in `qubit_op`, while also returning the `nuclear_repulsion_energy` and `core_electron_energy`.

In [4]:
def get_qubit_op(BL):
    # Define Molecule
    molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Li", "H"],
        coords=([0.0, 0.0, 0.0], [BL, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )
    
    # molecule = MoleculeInfo(
    #     # Coordinates in Angstrom
    #     symbols=["Be", "H", "H"],  # Beryllium and two hydrogens
    #     coords=([0.0, 0.0, 0.0], [BL, 0.0, 0.0], [-BL, 0.0, 0.0]),  # Linear geometry
    #     multiplicity=1,  # = 2*spin + 1, for singlet state
    #     charge=0,  # Neutral molecule
    # )

    driver = PySCFDriver.from_molecule(molecule)

    # Get properties
    properties = driver.run()

    # Now you can get the reduced electronic structure problem
    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)
    fermionic_op = problem.hamiltonian.second_q_op()
    
    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    mapper = ParityMapper(num_particles=problem.num_particles)
    tapered_mapper = problem.get_tapered_mapper(mapper)
    qubit_op = tapered_mapper.map(fermionic_op)

    core_electron_energy = problem.hamiltonian.constants['FreezeCoreTransformer']
    nuclear_repulsion_energy = problem.nuclear_repulsion_energy
    
    return qubit_op, num_particles, num_spatial_orbitals, problem, tapered_mapper, nuclear_repulsion_energy, core_electron_energy
    

In [5]:
(qubit_op, num_particles, num_spatial_orbitals, problem, mapper,nuclear_repulsion_energy, core_electron_energy) = get_qubit_op(BL)



In [6]:
# Extract terms and coefficients
pauli_list = list(zip(qubit_op.paulis, qubit_op.coeffs))

# Sort terms by absolute coefficient magnitude (largest first)
sorted_pauli_list = sorted(pauli_list, key=lambda x: abs(x[1]), reverse=True)

# Print sorted terms
# for pauli, coef in sorted_pauli_list:
#     print(f"Pauli: {pauli}, Coefficient: {coef}")

# Save the list
with open("sorted_pauli_list.pkl", "wb") as f:
    dill.dump(sorted_pauli_list, f)

In [7]:
dd = np.real(min(np.linalg.eigvals(qubit_op.to_matrix())))

### 2 Perform VQE

Define a class to perform VQE.

In [8]:
# Define the custom VQE
class CustomVQE:
    def __init__(self, estimator, ansatz, optimizer, qubit_op, dd, NREplusCEE, custom_function, initial_point=None):
        self.estimator = estimator
        self.ansatz = ansatz
        self.optimizer = optimizer
        self.qubit_op = qubit_op
        self.NREplusCEE = NREplusCEE
        self.dd = dd
        self.custom_function = custom_function
        self.initial_point = initial_point  or [0] * ansatz.num_parameters

    def compute_minimum_eigenvalue(self):
        # Define the function to minimize
        def objective_function(params):
            # Evaluate the expectation value of the qubit operator
            expectation = self.estimator.run([self.ansatz], [self.qubit_op], [params]).result().values[0]
            # Evaluate the custom objective function
            return self.custom_function(expectation, params)

        # Use the `minimize` method for optimization
        opt_result = self.optimizer.minimize(
            fun=objective_function,
            x0=self.initial_point
        )

        result = {
            'x': [float(value) for value in opt_result.x],
            'fun': opt_result.fun,
            'dd': self.dd,
            'NREplusCEE': self.NREplusCEE
        }

        
        return result


In [9]:
# l=12 and maxiter = 200 works for LiH and HF

In [10]:
optimizer = SLSQP(maxiter=300)
noiseless_estimator = Estimator(approximation=True)


# Define ansatz
init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
var_form = EfficientSU2(qubit_op.num_qubits, reps=12, entanglement='full', skip_unentangled_qubits=False, parameter_prefix='a')


# Example Custom Objective Function
def custom_objective_function(expectation_value, parameters):
    # penalty_term = 0.1 * sum(param**2 for param in parameters)  # Example regularization
    return expectation_value 


# Create an instance of CustomVQE
custom_vqe = CustomVQE(
    estimator=noiseless_estimator,
    ansatz=var_form,
    optimizer=optimizer,
    qubit_op=qubit_op,
    dd = dd,
    NREplusCEE = nuclear_repulsion_energy + core_electron_energy,
    custom_function=custom_objective_function,
    initial_point = list(np.random.uniform(-np.pi, np.pi, var_form.num_parameters))
)


In [11]:
vqe_result = custom_vqe.compute_minimum_eigenvalue()


In [12]:
len(vqe_result['x'])

156

In [13]:
print(vqe_result['fun'] - vqe_result['dd'])

5.435142647725044e-06


In [14]:
with open("vqe_result.pkl", "wb") as f:
    dill.dump(vqe_result, f)